In [1]:
import pandas as pd

In [195]:
rmat = pd.read_csv('../rs/resource/etext_10/etext_compos.txt',sep='\t',header=None)

In [196]:
rmat[:2]

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,file:/software/home/chenjh/mallet-2.0.8/etext/...,0.034328,0.003914,0.296675,0.144313,0.000084,0.021431,0.334450,0.006278,0.062092,0.096435
1,1,file:/software/home/chenjh/mallet-2.0.8/etext/...,0.010474,0.000850,0.044664,0.028078,0.000044,0.010328,0.016326,0.002303,0.862257,0.024676


In [197]:
rmat['e'] = rmat[1].apply(lambda x:x[-4:])    

In [198]:
rmat[:1]

,0,1,2,3,4,5,6,7,8,9,10,11,e
0,0,file:/software/home/chenjh/mallet-2.0.8/etext/...,0.034328,0.003914,0.296675,0.144313,0.000084,0.021431,0.33445,0.006278,0.062092,0.096435,e052


In [199]:
del rmat[0]
del rmat[1]

In [200]:
rmat = rmat.set_index('e')

In [201]:
rmat[:2]

,2,3,4,5,6,7,8,9,10,11
e,,,,,,,,,,
e052,0.034328,0.003914,0.296675,0.144313,0.000084,0.021431,0.334450,0.006278,0.062092,0.096435
e055,0.010474,0.000850,0.044664,0.028078,0.000044,0.010328,0.016326,0.002303,0.862257,0.024676


In [202]:
evmap = pd.read_csv('../rs/ev-mapping.txt',header=None,sep=' ')

In [203]:
evmap.rename(columns={0:'e',1:'v'},inplace=True)

In [204]:
rmat.reset_index(inplace=True)

In [205]:
vrmat = pd.merge(rmat,evmap,on='e')

In [206]:
vrmat = vrmat.set_index(['v','e'])

In [207]:
vmat = vrmat.groupby(level=0).mean()

In [208]:
vmat[:2]

,2,3,4,5,6,7,8,9,10,11
v,,,,,,,,,,
v001,0.077877,0.005772,0.177866,0.111784,0.000667,0.018326,0.257271,0.004757,0.235443,0.110237
v002,0.056601,0.005213,0.027520,0.056177,0.000693,0.012965,0.432048,0.002953,0.058417,0.347414


In [209]:
qmat = pd.read_csv('../rs/resource/etext_10/expqs_dt',sep='\t',header=None,skiprows=1)
qmat[:2]

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,pavlova,0.072985,0.025054,0.571895,0.035948,0.009804,0.011983,0.075163,0.027233,0.011983,0.157952
1,1,asian,0.095085,0.016026,0.456197,0.058761,0.011752,0.035256,0.120726,0.028846,0.037393,0.139957


In [210]:
qmap = pd.read_csv('../rs/resource/qmap',header=None,sep=' ')

In [211]:
qmap[:2]

,0,1
0,0,7011
1,1,7014


In [212]:
qmat = pd.merge(qmat,qmap,on=0)

In [213]:
del qmat['1_x']
del qmat[0]

In [214]:
qmat.rename(columns={'1_y':'q'},inplace=True)

In [215]:
from scipy.stats import entropy
import numpy as np
def cal_sim(inA, inB):
    return entropy(inA, inB)

In [216]:
qmat.set_index('q',inplace=True)

In [217]:
def get_sim_result(qmat, rmat):
    result = {}
    # t = []
    for i in xrange(len(qmat)):
        qresult = []
        qm = qmat.iloc[i].tolist()
        q = qmat.iloc[i].name
        for j in xrange(len(rmat)):
            rm = rmat.iloc[j].tolist()
            r = rmat.iloc[j].name
            score = cal_sim(qm, rm)
    #         t.append(Timer('cal_sim(qm, rm)','from __main__ import cal_sim,qm,rm').timeit(10))
            qresult.append((r,score))
        result[q] = qresult
    return result

In [218]:
qvsim = get_sim_result(qmat, vmat)

In [219]:
malletv = pd.DataFrame([(q,t[0],t[1]) for q,values in qvsim.iteritems() for t in values])

In [220]:
malletv.rename(columns={0:'q',1:'v',2:'score'},inplace=True)

In [221]:
def f_mk(df):
    df['r_mk'] = np.arange(1,len(df)+1)
    return df

def get_rank_by(df, col_name, ascending=False):
    group = df.groupby('q')[col_name]
    re = df.set_index(['q', df.index])
    re = re.ix[group.apply(lambda x:x.sort_values(ascending=ascending)).index]
    out = re.groupby(level=0).apply(f_mk).reset_index()
    return out

In [222]:
malletv = get_rank_by(malletv,'score', True)

In [223]:
fil = malletv.query("v=='v001'").take([0,-1],axis=1)

In [224]:
out = pd.merge(malletv,fil,on='q').query('r_mk_x<=r_mk_y')

In [225]:
out[:2]

,q,level_1,v,score,r_mk_x,r_mk_y
0,7011,1400,v009,0.332248,1,5
1,7011,1405,v014,0.454615,2,5


In [226]:
out['v_x'] = out['v'].apply(lambda x:'FW14-'+x)

In [227]:
out['runid'] = 'kapokvs'
out[:2]

,q,level_1,v,score,r_mk_x,r_mk_y,v_x,runid
0,7011,1400,v009,0.332248,1,5,FW14-v009,kapokvs
1,7011,1405,v014,0.454615,2,5,FW14-v014,kapokvs


In [228]:
out.take([0,-2,-1],axis=1).to_csv('resource/vs_mal10',sep=' ',header=None,index=False)

_________________

In [244]:
qmap = pd.read_csv('../rs/resource/qmap',header=None,sep=' ')

In [245]:
qmap[:2]

,0,1
0,0,7011
1,1,7014


In [240]:
vs = []
for i in xrange(1,25):
    v = 'v'+str(i).zfill(3)
    vs.append(v)

In [307]:
vmat = pd.read_csv('resource/vtext_10/vtext_compos.txt',sep='\t',header=None)

In [308]:
vmat['v'] = vs
del vmat[0]
del vmat[1]

In [309]:
vmat.set_index('v',inplace=True)
vmat[:2]

,2,3,4,5,6,7,8,9,10,11
v,,,,,,,,,,
v001,0.000868,0.054972,0.008038,0.765335,0.031689,0.005839,0.024181,0.016485,0.080317,0.012275
v002,0.002606,0.082238,0.006816,0.000249,0.017489,0.004855,0.097376,0.018363,0.036312,0.733695


In [310]:
vmat = vmat.apply(pd.to_numeric,errors='ignore')

In [311]:
qmat = pd.read_csv('resource/vtext_10/expqs_dt',sep='\t',header=None,skiprows=1)
qmat[:2]

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,pavlova,0.004390,0.492450,0.002998,0.015194,0.023002,0.028146,0.025021,0.033736,0.268524,0.106537
1,1,asian,0.002034,0.518675,0.002937,0.008083,0.061071,0.007169,0.049447,0.010378,0.331055,0.009151


In [312]:
qmat = pd.merge(qmat,qmap,on=0)

In [313]:
del qmat['1_x']
del qmat[0]
qmat.rename(columns={'1_y':'q'},inplace=True)
qmat.set_index('q',inplace=True)
qmat[:2]

,2,3,4,5,6,7,8,9,10,11
q,,,,,,,,,,
7011,0.004390,0.492450,0.002998,0.015194,0.023002,0.028146,0.025021,0.033736,0.268524,0.106537
7014,0.002034,0.518675,0.002937,0.008083,0.061071,0.007169,0.049447,0.010378,0.331055,0.009151


In [314]:
qmat = qmat.apply(pd.to_numeric,errors='ignore')

In [315]:
qvsim = get_sim_result(qmat, vmat)

In [316]:
malletv = pd.DataFrame([(q,t[0],t[1]) for q,values in qvsim.iteritems() for t in values])
malletv.rename(columns={0:'q',1:'v',2:'score'},inplace=True)
malletv = get_rank_by(malletv,'score', True)
fil = malletv.query("v=='v001'").take([0,-1],axis=1)
out = pd.merge(malletv,fil,on='q').query('r_mk_x<=r_mk_y')
out[:2]

,q,level_1,v,score,r_mk_x,r_mk_y
0,7011,1409,v018,0.589554,1,9
1,7011,1405,v014,0.835797,2,9


In [317]:
out['v_x'] = out['v'].apply(lambda x:'FW14-'+x)
out['runid'] = 'kapokvs'
out[:2]

,q,level_1,v,score,r_mk_x,r_mk_y,v_x,runid
0,7011,1409,v018,0.589554,1,9,FW14-v018,kapokvs
1,7011,1405,v014,0.835797,2,9,FW14-v014,kapokvs


In [318]:
out.take([0,-2,-1],axis=1).to_csv('resource/vs_bigmal/vs_mal10',sep=' ',header=None,index=False)